In [83]:
import pandas as pd
import numpy as np 
import ast
import json

CommunityToPoliceDistrict = []
with open('PoliceDistrictToCommunityNamesOutput.txt') as fp:
    for line in fp:
        communityNum, communityName, policeDist  = line.strip().split(":")
        communityName = communityName.strip()
        CommunityToPoliceDistrict.append({"Community Area": float(communityNum), "Community_Name": communityName,
                                         "Police_District": policeDist})

CommunityPoliceDistrictDF = pd.DataFrame(CommunityToPoliceDistrict)
CommunityPoliceDistrictDF.head()

,Community Area,Community_Name,Police_District
0,1.0,ROGERS PARK,24
1,2.0,WEST RIDGE,24
2,3.0,UPTOWN,23
3,4.0,LINCOLN SQUARE,20
4,5.0,NORTH CENTER,19


In [111]:
CrimeData = pd.read_csv("Crimes2001_2020.csv",
                       dtype={'ID': int, 'Case Number' : str, 'Date' : str, 'Block' : str, 
                             'IUCR' : str, 'Primary Type' : str, 'Description' : str, 
                             'Location Description' : str, 'Arrest' : str, 'Domestic' : str, 'Beat' : str, 
                             'District' : float, 'Ward' : float, 'Community Area' : float, 'FBI Code' : str, 
                              'X Coordinate' : str, 'Y Coordinate' : str, 'Year' : int,'Updated On' : str,
                             'Latitude' : str, 'Longitude' : str, 'Location' : str})

CrimeData = CrimeData.drop(['X Coordinate', 'Y Coordinate', 'Latitude', 'Longitude', 'Location'], axis = 1)
First100CrimeData = CrimeData[:100]
First100CrimeData = First100CrimeData.join(CommunityPoliceDistrictDF.set_index('Community Area'), on='Community Area')
print(First100CrimeData.columns)

999
Index(['ID', 'Case Number', 'Date', 'Block', 'IUCR', 'Primary Type',
       'Description', 'Location Description', 'Arrest', 'Domestic', 'Beat',
       'District', 'Ward', 'Community Area', 'FBI Code', 'Year', 'Updated On',
       'Community_Name', 'Police_District'],
      dtype='object')


In [85]:
# Used to Extract the data from the Socio Economic CSV File 
# To train our program what specific communities are better or worse off than average
import pandas as pd 
import numpy as np 

# We can use .describe() to find the benchmark levels of poverty per community. Compare community to the average and give weightage 
columns = [
'Community Area',
'community_area_name', 
'%_housing_crowded', 
'%_households_below_poverty', 
'%_age16_unemployed',
'%_age25_nohighschool',
'%_not_working_age',
'per_capita_income',
'hardship_index']

SocioeconomicData = pd.read_csv("SocioeconomicData.csv")
SocioeconomicData.columns = columns

First100CrimeData = First100CrimeData.join(SocioeconomicData.set_index('Community Area'), on='Community Area')

print(First100CrimeData.columns)
print(First100CrimeData.head())

Index(['ID', 'Case Number', 'Date', 'Block', 'IUCR', 'Primary Type',
       'Description', 'Location Description', 'Arrest', 'Domestic', 'Beat',
       'District', 'Ward', 'Community Area', 'FBI Code', 'Year', 'Updated On',
       'Community_Name', 'Police_District', 'community_area_name',
       '%_housing_crowded', '%_households_below_poverty', '%_age16_unemployed',
       '%_age25_nohighschool', '%_not_working_age', 'per_capita_income',
       'hardship_index'],
      dtype='object')
         ID Case Number           Date                 Block  IUCR  \
0  11034701    JA366925   1/1/01 11:00       016XX E 86TH PL  1153   
1  11227287    JB147188   10/8/17 3:00    092XX S RACINE AVE   281   
2  11227583    JB147595  3/28/17 14:00       026XX W 79TH ST   620   
3  11227293    JB147230   9/9/17 20:17  060XX S EBERHART AVE   810   
4  11227634    JB147599  8/26/17 10:00   001XX W RANDOLPH ST   281   

          Primary Type                          Description  \
0   DECEPTIVE PRACTICE  

In [86]:
import pandas as pd

census = pd.read_csv("census_data.csv", header=1)

CensusData = pd.DataFrame(census)
CensusData = CensusData[CensusData.columns[range(67)]]
CensusData = CensusData.rename(columns={"Geog": "Community Name", "GeogKey": "Community Area"})

# First100CrimeData = First100CrimeData.join(CensusData.set_index('Community Area'), on='Community Area')
First100CrimeData = First100CrimeData.merge(CensusData, on='Community Area', how='left')
First100CrimeData = First100CrimeData.drop(['community_area_name','Community_Name'] , axis =1)
for col in First100CrimeData.columns:
    print(col)

print(First100CrimeData.head())

ID
Case Number
Date
Block
IUCR
Primary Type
Description
Location Description
Arrest
Domestic
Beat
District
Ward
Community Area
FBI Code
Year
Updated On
Police_District
%_housing_crowded
%_households_below_poverty
%_age16_unemployed
%_age25_nohighschool
%_not_working_age
per_capita_income
hardship_index
Community Name
Total Population
Not Hispanic or Latino, White alone
Not Hispanic or Latino, Black or African American alone
Not Hispanic or Latino, American Indian and Alaska Native alone
Not Hispanic or Latino, Asian alone
Not Hispanic or Latino, Native Hawaiian and Other Pacific Islander alone
Not Hispanic or Latino, Some Other Race alone
Not Hispanic or Latino, Two or More Races
Hispanic or Latino
Male: Under 5 years old
Male: 5 to 9 years
Male: 10 to 14 years
Male: 15 to 17 years
Male: 18 and 19 years
Male: 20 years
Male: 21 years
Male: 22 to 24 years
Male: 25 to 29 years
Male: 30 to 34 years
Male: 35 to 39 years
Male: 40 to 44 years
Male: 45 to 49 years
Male: 50 to 54 years
Male: 55

In [96]:
# Function to format datetime for the weather API
def format_datetime(string):
    date, time = string.split()

    # Format date
    month,day,year = date.split('/')
    if len(month) == 1:
        month = '0' + month
    if len(day) == 1:
        day = '0' + day
    year = '20' + year
    formatted_date = year + '-' + month + '-' + day

    # Format time
    formatted_start_time = 'T' + time + ':00'
    formatted_end_time = 'T' + time + ':59'

    formatted_start_datetime = formatted_date + formatted_start_time
    formatted_end_datetime = formatted_date + formatted_end_time

    return formatted_start_datetime,formatted_end_datetime

In [107]:
import requests
import io

# Call to weather API function
def getWeather(community_name, start, end):
    url = "https://visual-crossing-weather.p.rapidapi.com/history"

    querystring = {"startDateTime":start,"aggregateHours":"1","location": community_name + " IL","endDateTime":end,"unitGroup":"us"}
    headers = {
        'x-rapidapi-host': "visual-crossing-weather.p.rapidapi.com",
        'x-rapidapi-key': "fcec7c8393mshb733d50fc42d667p1a997fjsn531e02e32408"
        }

    response = requests.request("GET", url, headers=headers, params=querystring)
    print(response.text)
    # Put response into a dataframe
    rawData = pd.read_csv(io.StringIO(response.text))
    return rawData


In [110]:
for index, row in First100CrimeData.head().iterrows():
    start, end = format_datetime(row['Date'])
    community_area = row['Community Name']
    print(community_area)
    df = getWeather(community_area, start, end)
    df.head()

Avalon Park
Address,Date time,Minimum Temperature,Maximum Temperature,Temperature,Dew Point,Relative Humidity,Heat Index,Wind Speed,Wind Gust,Wind Direction,Wind Chill,Precipitation,Precipitation Cover,Snow Depth,Visibility,Cloud Cover,Sea Level Pressure,Weather Type,Latitude,Longitude,Resolved Address,Name,Info,Conditions
"Avalon Park IL",01/01/2001 11:00:00,23.1,23.1,23.1,14.1,67.98,,8.1,,320,14,0,0,,10,,1031.4,"",41.74507,-87.58816,"Avalon Park, Chicago, IL, United States","","","Clear"

Washington Heights
Address,Date time,Minimum Temperature,Maximum Temperature,Temperature,Dew Point,Relative Humidity,Heat Index,Wind Speed,Wind Gust,Wind Direction,Wind Chill,Precipitation,Precipitation Cover,Snow Depth,Visibility,Cloud Cover,Sea Level Pressure,Weather Type,Latitude,Longitude,Resolved Address,Name,Info,Conditions
"Washington Heights IL",10/08/2017 03:00:00,57.1,57.1,57.1,54.8,91.99,,6.7,,220,,0,0,,10,0,1007.3,"",41.71584,-87.64884,"Washington Heights, Chicago, IL, United States","",